# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [2]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, desc, explode, lit, min, max, split, udf, isnull, count, sum as Fsum
from pyspark.sql.types import IntegerType, DateType

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, DecisionTreeClassifier, NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import re
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [3]:
# create a Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("Identify Features") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [4]:
df = spark.read.json('mini_sparkify_event_data.json')

df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [7]:
df.head(3)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30'),
 Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9'),
 Row(artist='Adam Lambert', auth='Logged In', firstName='Colin', gender='M', itemInSession=51, lastName='Freeman', length=282.8273, level='paid', location='

In [8]:
#Number of rows before cleaning
df.count()

286500

In [9]:
#Drop missing values
df = df.dropna(how = "any", subset = ["userId", "sessionId"])
df.count()

286500

In [10]:
#Remove rows with empty userId
df = df.filter(df.userId != "")
df.count()

278154

In [11]:
#Number of users
df.select("userId").dropDuplicates().count()

225

In [12]:
# Number of sessions
df.select("sessionId").dropDuplicates().count()

2312

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [ ]:
df.select("page").dropDuplicates().show()

In [ ]:
# create a list for churn users
churn_user_df = df.filter(df.page=="Cancellation Confirmation").select("userId").dropDuplicates()
churn_user_list = [user["userId"] for user in churn_user_df.collect()]

In [ ]:
# create a column for churn
df_withchurn = df.withColumn("churn", df.userId.isin(churn_user_list))

In [ ]:
df_withchurn.dropDuplicates(["userId", "gender"]).groupby(["churn", "gender"]).count().sort("churn").show()

In [ ]:
df_pd = df_withchurn.dropDuplicates(["userId", "gender"]).groupby(["churn", "gender"]).count().sort("churn").toPandas()
sns.barplot(x='churn', y='count', hue='gender', data=df_pd)

In [ ]:
churn_count = df_withchurn.filter(df_withchurn.churn==1).count()
no_churn_count = df_withchurn.filter(df_withchurn.churn==0).count()

df_temp1 = df_withchurn.filter(df_withchurn.churn==1).filter(df_withchurn.page!="NextSong").groupby(["page"]).count()
df_temp1 = df_temp1.withColumn("cFreq", df_temp1["count"]/(churn_count/100)).sort("page")

df_temp2 = df_withchurn.filter(df_withchurn.churn==0).filter(df_withchurn.page!="NextSong").groupby(["page"]).count()
df_temp2 = df_temp2.withColumn("ncFreq", df_temp2["count"]/(no_churn_count/100)).sort("page")

In [ ]:
df_pd = df_temp1.join(df_temp2, "page", "outer").drop("count").fillna(0).toPandas()
df_pd.plot.bar("page")

In [ ]:
df_pd = df_withchurn.filter(df_withchurn.page=="Cancellation Confirmation").groupby("level").count().toPandas()
sns.barplot(x="level", y="count", data=df_pd)

In [ ]:
df_valid_songs = df_withchurn.where(df_withchurn.song!='null').groupby(['churn','userId']) \
    .agg(count(df_withchurn.song).alias('SongsPlayed')).orderBy('churn').toPandas()

In [ ]:
df_valid_songs.hist(['SongsPlayed'], by='churn', sharex=True, sharey=True,density=1)
plt.plot()

In [ ]:
#investigate the average numbers of thumps-up and thumps-down by two groups.
df_valid_thumbsup = df_withchurn.where(df_withchurn.page=='Thumbs Up') \
    .groupby(['churn','userId']).agg(count(col('page')).alias('ThumbsUps')).orderBy('churn').toPandas()
df_valid_thumbsup.hist('ThumbsUps', by='churn', bins=20, sharex=True, sharey=True, density=1)

In [ ]:
print('ThumbsUps for people who stay',df_valid_thumbsup[df_valid_thumbsup['churn']==False]['ThumbsUps'].mean())
print('ThumbsUps for people who leave',df_valid_thumbsup[df_valid_thumbsup['churn']==True]['ThumbsUps'].mean())

In [ ]:
df_valid_thumbsdown = df_withchurn.where(df_withchurn.page=='Thumbs Down') \
    .groupby(['churn','userId']).agg(count(col('page')).alias('ThumbsDowns')).orderBy('churn').toPandas()
df_valid_thumbsdown.hist('ThumbsDowns', by='churn', bins=20, sharex=True, sharey=True, density=1)

In [ ]:
print('ThumbsDowns for people who stay',df_valid_thumbsdown[df_valid_thumbsup['churn']==False]['ThumbsDowns'].mean())
print('ThumbsDowns for people who leave',df_valid_thumbsdown[df_valid_thumbsup['churn']==True]['ThumbsDowns'].mean())

In [ ]:
get_day = udf(lambda x: datetime.datetime.fromtimestamp(x/1000), DateType())

In [ ]:
song=udf(lambda x : int(x=='NextSong'), IntegerType())
home_visit=udf(lambda x : int(x=='Home'), IntegerType())
df_withchurn = df_withchurn.withColumn('date', get_day(col('ts')))

In [ ]:
#difference in number of songs played between users who churned and didnt?
df_withchurn.filter(col('churn')==1).withColumn('songPlayed', song(col('page'))).agg({'songPlayed':'mean'}).show()
df_withchurn.filter(col('churn')==0).withColumn('songPlayed', song(col('page'))).agg({'songPlayed':'mean'}).show()

In [ ]:
#number of songs couldn't be played due to errors
df_withchurn.filter((df_withchurn.page=='Error')&(df_withchurn.churn==1)).select('userId', 'page', 'ts', 'length')\
    .withColumn('date', get_day(col('ts')))\
    .groupBy('userId', 'date').agg({'page':'count'}).select('count(page)').describe().show()

df_withchurn.filter((df_withchurn.page=='Error')&(df_withchurn.churn==0)).select('userId', 'page', 'ts', 'length')\
    .withColumn('date', get_day(col('ts')))\
    .groupBy('userId', 'date').agg({'page':'count'}).select('count(page)').describe().show()

In [ ]:
#ratio of those who cancelled subscriptions both free and paid
print(df_withchurn.filter((df_withchurn.page=='Cancellation Confirmation') & (df_withchurn.level=='paid')).count(),
df_withchurn.filter((df.page=='Cancellation Confirmation') & (df.level=='free')).count())

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [ ]:
#Gender
df_gender = df_withchurn.dropDuplicates(['userId']).sort('userId').select(['userId','gender'])
df_gender = df_gender.replace(['F','M'], ['1', '0'], 'gender')
df_gender.withColumn('gender', df_gender.gender.cast("int"))

In [ ]:
df_gender.show()

In [ ]:
#number of days since registration
user_max_ts = df_withchurn.groupby("userId").max("ts").sort("userId")
user_reg_ts = df_withchurn.select("userId", "registration").dropDuplicates().sort("userId")
user_reg_days = user_reg_ts.join(user_max_ts, user_reg_ts.userId == user_max_ts.userId).select(user_reg_ts["userId"], ((user_max_ts["max(ts)"]-user_reg_ts["registration"])/(1000*60*60*24)).alias("regDay"))

In [ ]:
user_reg_days.show()

In [ ]:
#Number of songs played
df_songs = df_withchurn.where(df_withchurn.song!='null').groupby('userId') \
    .agg(count(df_withchurn.song).alias('SongsPlayed')).orderBy('userId') \
    .select(['userId','SongsPlayed'])

In [ ]:
df_songs.show()

In [ ]:
#Average time per session
user_session_time = df_withchurn.groupby("userId", "sessionId").agg(((max(df_withchurn.ts)-min(df_withchurn.ts))/(1000*60)).alias("sessionTime"))
user_session_time_stat = user_session_time.groupby("userId").agg(avg(user_session_time.sessionTime).alias("avgSessionTime")).sort("userId")

In [ ]:
user_session_time_stat.show()

In [ ]:
#pick the number of singers user has listened
df_singers = df_withchurn.dropDuplicates(['userId','artist']).groupby('userId') \
    .agg(count(df_withchurn.artist).alias('SingersListened')).orderBy('userId') \
    .select(['userId','SingersListened'])

In [ ]:
df_singers.show()

In [ ]:
#Number of songs per session
user_session_songs = df_withchurn.filter(df_withchurn.page=="NextSong").groupby("userId", "sessionId").count()
user_session_songs_avg = user_session_songs.groupby("userId").agg(avg(user_session_songs["count"]).alias("avgSessionSongs")).sort("userId")

In [ ]:
user_session_songs_avg.show()

In [ ]:
#Number of sessions
user_session_count = df_withchurn.select("userId", "sessionId").dropDuplicates().groupby("userId").count()
user_session_count = user_session_count.withColumnRenamed("count", "sessionCount").sort('userID')

In [ ]:
user_session_count.show()

In [ ]:
#Number of friends
numFriends = df_withchurn.filter(df_withchurn.page=='Add Friend')\
        .select('userId', 'page')\
        .groupBy('userId').count().withColumnRenamed('count', 'numFriends').sort('userId')

In [ ]:
numFriends.show()

In [ ]:
df_final = df_withchurn.dropDuplicates(['userId']).sort('userId').select(['userId','churn'])
for feature in [df_gender, user_reg_days, df_songs, user_session_time_stat, df_singers, user_session_songs_avg,user_session_count, numFriends]:
    df_final = df_final.join(feature,'userId')

In [ ]:
df_final.show()

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [ ]:
df_final.groupby("churn").count().show()

In [ ]:
#df_final.write.save('df_final.csv', format="csv", header=True)

In [ ]:
df_final = spark.read.csv('df_final.csv', header=True)

In [ ]:
num_features_list = df_final.columns[1:]
for f in num_features_list:
    f_name = f + "Num"
    df_final = df_final.withColumn(f_name, df_final[f].cast("float"))
    df_final = df_final.drop(f)

In [ ]:
df_final.dtypes

In [ ]:
df_final.show(5)

In [ ]:
assembler = VectorAssembler(inputCols=df_final.columns[1:-1], outputCol="NumFeatures")
data = assembler.transform(df_final)

In [ ]:
scaler = StandardScaler(inputCol="NumFeatures", outputCol="ScaledNumFeatures", withStd=True)
scalerModel = scaler.fit(data)
data = scalerModel.transform(data)

In [ ]:
data = data.select(data.churnNum.alias("label"), data.ScaledNumFeatures.alias("features"))

In [ ]:
train, validation = data.randomSplit([0.9, 0.1], seed=42)
train = train.cache()

In [ ]:
lr =  LogisticRegression()
paramGrid = ParamGridBuilder() \
    .addGrid(lr.elasticNetParam,[0.0, 0.1, 0.5, 1.0]) \
    .addGrid(lr.regParam,[0.0, 0.05, 0.1]) \
    .build()

crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=3)
cvModel_lr = crossval.fit(train)

In [ ]:
cvModel_lr.save('cvModel_lr.model')

In [ ]:
cvModel_lr.avgMetrics

In [ ]:
dt = DecisionTreeClassifier()
paramGrid = ParamGridBuilder() \
    .addGrid(dt.impurity,['entropy', 'gini']) \
    .addGrid(dt.maxDepth,[2, 3, 4, 5, 6, 7, 8]) \
    .build()

crossval_dt = CrossValidator(estimator=dt,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=3)

cvModel_dt = crossval_dt.fit(train)

In [ ]:
cvModel_dt.save('cvModel_dt.model')

In [ ]:
cvModel_dt.avgMetrics

In [ ]:
stratified_train = train.sampleBy('label', fractions={0: 99/349, 1: 1.0}).cache()
stratified_train.groupby("label").count().show()

In [ ]:
lrs =  LogisticRegression()
paramGrid = ParamGridBuilder() \
    .addGrid(lrs.elasticNetParam,[0.0, 0.1, 0.5, 1.0]) \
    .addGrid(lrs.regParam,[0.0, 0.05, 0.1]) \
    .build()

crossval_lrs = CrossValidator(estimator=lrs,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=3)
cvModel_lrs = crossval_lrs.fit(stratified_train)
cvModel_lrs.avgMetrics

In [ ]:
cvModel_lrs.save('cvModel_lrs.model')

In [ ]:
results = cvModel_lr.transform(validation)

In [ ]:
tp = results.filter("label = 1 and prediction = 1").count()
fp = results.filter("label = 0 and prediction = 1").count()
fn = results.filter("label = 1 and prediction = 0").count()
precision = tp / (tp + fp) 
recall = tp / (tp + fn)
f1 = 2*precision*recall / (precision+recall)
print(precision)
print(recall)
print(f1)

In [ ]:
results = cvModel_lrs.transform(validation)

In [ ]:
tp = results.filter("label = 1 and prediction = 1").count()
fp = results.filter("label = 0 and prediction = 1").count()
fn = results.filter("label = 1 and prediction = 0").count()
precision = tp / (tp + fp) 
recall = tp / (tp + fn)
f1 = 2*precision*recall / (precision+recall)
print(precision)
print(recall)
print(f1)

In [ ]:
cvModel_lrs.bestModel.coefficients

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.